<a href="https://colab.research.google.com/github/Namrahh/AI-Projects/blob/main/LLM_Assignment_Search_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Setting up the google colab**

In [ ]:
from google.colab import files

# Upload your PDF file
uploaded = files.upload()


Saving fcomm-09-1465178.pdf to fcomm-09-1465178.pdf
Saving fcomm-09-1456509.pdf to fcomm-09-1456509.pdf
Saving media__entertainment_5086_nieuw.aa6e53fd64e3.pdf to media__entertainment_5086_nieuw.aa6e53fd64e3 (2).pdf


# **2. Extract text from PDF**

In [ ]:
!pip install PyPDF2

In [ ]:
import PyPDF2

# List of PDF paths (the keys of the uploaded files)
pdf_paths = list(uploaded.keys())

# Extract text from each PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

# Dictionary to store extracted text from each PDF
pdf_texts = {}

# Loop through each uploaded PDF file
for pdf_path in pdf_paths:
    pdf_texts[pdf_path] = extract_text_from_pdf(pdf_path)
    print(f"Extracted Text from {pdf_path} (sample):", pdf_texts[pdf_path][:500])  # Print a small sample of the extracted text

# Optionally, if you want to combine the text from all PDFs into one large corpus
combined_text = "\n".join(pdf_texts.values())

# Now you can continue with your further processing using `pdf_texts` or `combined_text`


Extracted Text from fcomm-09-1465178.pdf (sample): Frontiers in Communication 01 frontiersin.org
Ethics and journalistic challenges 
in the age of artificial intelligence: 
talking with professionals and 
experts
Beatriz Gutiérrez-Caneda 1*, Carl-Gustav Lindén 2 and 
Jorge Vázquez-Herrero 1
1 Departamento de Ciencias da Comunicación, Universidade de Santiago de Compostela, Santiago de 
Compostela, Spain, 2 Department of Information Science and Media Studies, University of Bergen, 
Bergen, Norway
The rapid advancement of artificial intelligence (
Extracted Text from fcomm-09-1456509.pdf (sample): Frontiers in Communication 01 frontiersin.org
The anthropomorphic pursuit of 
AI-generated journalistic texts: 
limits to expressing subjectivity
Cristian González-Arias 1,2*, Eirini Chatzikoumi 3 and 
Xosé López-García 1
1 Departamento de Ciencias de la Comunicación, Universidad de Santiago de Compostela, Santiago 
de Compostela, Spain, 2 Instituto de Literatura y Ciencias del Lenguaje, Pontif

# **3. Split the text into smaller documents**

In [ ]:
# Generate chunks from the PDF text
# Concatenate all text into a single string before splitting
all_text = " ".join(pdf_texts.values())

def split_text_into_chunks(text, chunk_size=1000):
    """Splits the text into chunks of a specified size.

    Args:
        text: The text to split.
        chunk_size: The desired size of each chunk. Defaults to 1000.

    Returns:
        A list of text chunks.
    """
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

chunks = list(split_text_into_chunks(all_text))
print("Number of Chunks:", len(chunks))
print("Sample Chunk:", chunks[0])

Number of Chunks: 164
Sample Chunk: Frontiers in Communication 01 frontiersin.org
Ethics and journalistic challenges 
in the age of artificial intelligence: 
talking with professionals and 
experts
Beatriz Gutiérrez-Caneda 1*, Carl-Gustav Lindén 2 and 
Jorge Vázquez-Herrero 1
1 Departamento de Ciencias da Comunicación, Universidade de Santiago de Compostela, Santiago de 
Compostela, Spain, 2 Department of Information Science and Media Studies, University of Bergen, 
Bergen, Norway
The rapid advancement of artificial intelligence (AI) is transforming the media 
industry by automating processes, with applications in data analysis, automated 
writing, format transformation, content personalization, and fact-checking. 
While AI integration offers new opportunities in journalism, it also raises ethical 
concerns around data privacy, algorithmic biases, transparency, and potential job 
displacement. This study employed qualitative interviews with media professionals 
and researchers to explo

# **4. Preprocess text**

In [ ]:
from gensim.utils import simple_preprocess

# Preprocess the chunks
preprocessed_corpus = [simple_preprocess(chunk) for chunk in chunks]
print("Preprocessed Corpus Sample:", preprocessed_corpus[:2])  # Print first two processed chunks


Preprocessed Corpus Sample: [['frontiers', 'in', 'communication', 'frontiersin', 'org', 'ethics', 'and', 'journalistic', 'challenges', 'in', 'the', 'age', 'of', 'artificial', 'intelligence', 'talking', 'with', 'professionals', 'and', 'experts', 'beatriz', 'gutiérrez', 'caneda', 'carl', 'gustav', 'lindén', 'and', 'jorge', 'vázquez', 'herrero', 'departamento', 'de', 'ciencias', 'da', 'comunicación', 'universidade', 'de', 'santiago', 'de', 'compostela', 'santiago', 'de', 'compostela', 'spain', 'department', 'of', 'information', 'science', 'and', 'media', 'studies', 'university', 'of', 'bergen', 'bergen', 'norway', 'the', 'rapid', 'advancement', 'of', 'artificial', 'intelligence', 'ai', 'is', 'transforming', 'the', 'media', 'industry', 'by', 'automating', 'processes', 'with', 'applications', 'in', 'data', 'analysis', 'automated', 'writing', 'format', 'transformation', 'content', 'personalization', 'and', 'fact', 'checking', 'while', 'ai', 'integration', 'offers', 'new', 'opportunities', 'i

# **5. Train word embeddings**

In [ ]:
from gensim.models import Word2Vec

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=preprocessed_corpus, vector_size=100, window=5, min_count=1, workers=4)
print("Word2Vec Model Trained")


Word2Vec Model Trained


# **6. Implement vector search**

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Generate document vectors by averaging word embeddings
def compute_doc_vector(doc, model):
    vectors = [model.wv[word] for word in doc if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

# Generate vectors for all chunks
document_vectors = [compute_doc_vector(doc, word2vec_model) for doc in preprocessed_corpus]

# Search Function
def search(query, model, document_vectors, original_chunks):
    # Preprocess and vectorize query
    query_vector = compute_doc_vector(simple_preprocess(query), model)

    # Compute cosine similarity
    similarities = cosine_similarity([query_vector], document_vectors)

    # Rank results
    ranked_indices = np.argsort(similarities[0])[::-1]

    # Retrieve results
    results = [(original_chunks[i], similarities[0][i]) for i in ranked_indices[:5]]  # Top 5 results
    return results

# Test the search
query = "What is deep learning?"
results = search(query, word2vec_model, document_vectors, chunks)
print("Search Results:")
for result in results:
    print(result)


Search Results:
('society, this does not mean that people truly understand what it entails. Moreover, this also applies to journalists and newsrooms. “I think the biggest challenge (….) is to really understand what artificial intelligence is. And that means at a more technical level how it works, what it actually does and therefore also understanding what its limits are, what its dangers are and at a high level what it is” , affirmed one interviewee. This professional and researcher also notes that, in general, we do not have a proper understanding of AI. He emphasizes that we are approaching the end of the hype cycle, and we will likely face disappointment because we expect AI to accomplish things beyond its capabilities. This concern about how journalists’ understanding of AI is closely related to how the media represents this technology. Newsrooms have an important role in building social imaginaries, and due to this, they have major responsibility regarding this issue: “This is the

# **7. Fine-tune a transfer model**

In [ ]:
!pip install datasets

In [ ]:
!pip install datasets transformers

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming you have your dataset in `custom_dataset`
# Define custom_dataset first
custom_dataset = Dataset.from_dict({"text": chunks, "label": [0] * len(chunks)})
# Get the indices for the train/val split
indices = np.arange(len(custom_dataset))
train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=42)  # 80-20 split, adjust as necessary

# Create the train and validation datasets using the indices
train_data = custom_dataset.select(train_indices)
val_data = custom_dataset.select(val_indices)

In [ ]:
print(train_data)
print(val_data)

Dataset({
    features: ['text', 'label'],
    num_rows: 131
})
Dataset({
    features: ['text', 'label'],
    num_rows: 33
})


In [ ]:
print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")

Training samples: 131
Validation samples: 33


In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments,  EvalPrediction, EarlyStoppingCallback

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = custom_dataset.map(tokenize_function, batched=True)

# Define compute_metrics function
def compute_metrics(eval_pred: EvalPrediction):
  """
  Calculates and returns a dictionary of metrics.

  Args:
    eval_pred: An EvalPrediction object containing predictions and labels.

  Returns:
    A dictionary of metrics, e.g., {'accuracy': 0.85}.
  """
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  accuracy = np.mean(predictions == labels)
  return {'accuracy': accuracy}

# Load pre-trained model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Tokenized train and validation datasets
tokenized_train = train_data.map(tokenize_function, batched=True)
tokenized_val = val_data.map(tokenize_function, batched=True)

# Set format for Trainer compatibility
tokenized_train = tokenized_train.with_format("torch")
tokenized_val = tokenized_val.with_format("torch")

training_args = TrainingArguments(
    output_dir="./results",               # Output directory
    evaluation_strategy="epoch",          # Evaluate every epoch
    save_strategy="epoch",                # Save model after every epoch
    num_train_epochs=5,                   # Number of epochs
    per_device_train_batch_size=4,        # Batch size
    per_device_eval_batch_size=4,         # Eval batch size
    save_total_limit=2,                   # Limit the number of saved models
    load_best_model_at_end=True,          # Load the best model based on validation loss
    metric_for_best_model="eval_loss",    # Metric to choose the best model
    greater_is_better=False,              # Lower loss is better
    logging_dir='./logs',                 # Directory to save logs
    logging_steps=100,                    # Frequency of logging
    weight_decay=0.01,                    # Add weight decay (regularization)
    warmup_steps=500,                     # Learning rate warmup
    report_to="none",                     # Disable wandb logging
    disable_tqdm=True                     # Disable progress bar
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)

trainer.train()


Map:   0%|          | 0/164 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/131 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


{'eval_loss': 0.4698374569416046, 'eval_accuracy': 1.0, 'eval_runtime': 1.033, 'eval_samples_per_second': 31.946, 'eval_steps_per_second': 8.713, 'epoch': 1.0}
{'eval_loss': 0.09122232347726822, 'eval_accuracy': 1.0, 'eval_runtime': 1.0368, 'eval_samples_per_second': 31.829, 'eval_steps_per_second': 8.681, 'epoch': 2.0}
{'eval_loss': 0.006198262330144644, 'eval_accuracy': 1.0, 'eval_runtime': 1.0786, 'eval_samples_per_second': 30.595, 'eval_steps_per_second': 8.344, 'epoch': 3.0}
{'loss': 0.3202, 'grad_norm': 0.21348340809345245, 'learning_rate': 1e-05, 'epoch': 3.0303030303030303}
{'eval_loss': 0.001715980819426477, 'eval_accuracy': 1.0, 'eval_runtime': 1.0753, 'eval_samples_per_second': 30.689, 'eval_steps_per_second': 8.37, 'epoch': 4.0}
{'eval_loss': 0.000926994311157614, 'eval_accuracy': 1.0, 'eval_runtime': 1.0632, 'eval_samples_per_second': 31.039, 'eval_steps_per_second': 8.465, 'epoch': 5.0}
{'train_runtime': 145.4071, 'train_samples_per_second': 4.505, 'train_steps_per_second

TrainOutput(global_step=165, training_loss=0.19522744516531626, metrics={'train_runtime': 145.4071, 'train_samples_per_second': 4.505, 'train_steps_per_second': 1.135, 'train_loss': 0.19522744516531626, 'epoch': 5.0})

In [ ]:
# Evaluate the model on the validation set using the tokenized validation data
results = trainer.evaluate(eval_dataset=tokenized_val)

# Print the evaluation results
print(f"Validation Results: {results}")

{'eval_loss': 0.000926994311157614, 'eval_accuracy': 1.0, 'eval_runtime': 1.043, 'eval_samples_per_second': 31.639, 'eval_steps_per_second': 8.629, 'epoch': 5.0}
Validation Results: {'eval_loss': 0.000926994311157614, 'eval_accuracy': 1.0, 'eval_runtime': 1.043, 'eval_samples_per_second': 31.639, 'eval_steps_per_second': 8.629, 'epoch': 5.0}


# **8. GPT-4 for Summarization**

In [ ]:
!pip install update gpt4all

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.3 MB/s eta 0:00:00


In [ ]:
from gpt4all import GPT4All
model = GPT4All("Meta-Llama-3-8B-Instruct.Q4_0.gguf") # downloads / loads a 4.66GB LLM
with model.chat_session():
    print(model.generate("How can I run LLMs efficiently on my laptop?", max_tokens=1024))

Downloading: 100%|██████████| 4.66G/4.66G [01:10<00:00, 66.5MiB/s]
Verifying: 100%|██████████| 4.66G/4.66G [00:08<00:00, 533MiB/s]


Large Language Models (LLMs) are powerful AI models that require significant computational resources to train and use. Running them efficiently on your laptop requires a combination of hardware, software, and optimization techniques. Here's a comprehensive guide to help you get started:

**Hardware Requirements:**

1. **CPU:** A modern CPU with multiple cores is essential for LLMs. Intel Core i5 or i7 (or AMD equivalent) are good starting points.
2. **GPU:** A dedicated graphics card can significantly accelerate computations. NVIDIA GeForce GTX 1060 or higher, or AMD Radeon RX 580 or higher, are recommended.
3. **RAM:** Ensure you have at least 16 GB of RAM to handle the model's memory requirements.

**Software Requirements:**

1. **Python:** LLMs typically rely on Python as their primary programming language. Install Python (version 3.x) and a compatible IDE like PyCharm, Visual Studio Code, or Jupyter Notebook.
2. **LLM Framework:** Choose an LLM framework that supports your laptop's

In [ ]:
# Summarize search results
def summarize_results(results):
    relevant_text = " ".join([result[0] for result in results])

    # Load the GPT4All model here within the function
    from gpt4all import GPT4All
    gpt4_model = GPT4All("Meta-Llama-3-8B-Instruct.Q4_0.gguf") # Re-load the model here
    with gpt4_model.chat_session(): # Using gpt4_model instead of model
        summary = gpt4_model.generate(f"Summarize this: {relevant_text}", max_tokens=1024) # Using gpt4_model instead of model
    return summary


# Example dummy results for testing
results = [("This is the first document text.",), ("This is the second document text.",)]

# Get and summarize search results
summary = summarize_results(results)
print("Summary:", summary)

Summary: This appears to be a brief summary of two documents:

1. The first document contains some general information or introduction.
2. The second document also has some content, but it's not specified what that might be.

If you'd like me to help with anything else, feel free to ask!


# **9. Error handling for PDF extraction ( extra step )**

In [ ]:
def extract_text_from_pdf(pdf_path):
    try:
        with open(pdf_path, "rb") as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text += page.extract_text()
            if not text:
                raise ValueError("No text found in the PDF")
    except Exception as e:
        print(f"Error: {e}")
        text = ""
    return text


# **10. Final Workflow**

In [ ]:
def custom_search_engine(query):
    # Search relevant chunks
    results = search(query, word2vec_model, document_vectors, chunks)

    # Summarize results using GPT-4
    summary = summarize_results(results)
    return summary

# Testing the workflow
query = "Explain the role of AI in media."
response = custom_search_engine(query)
print("Response:", response)


Response: The article discusses the integration of Artificial Intelligence (AI) in newsrooms and its implications for journalists and media outlets. The authors argue that while AI can bring efficiency, productivity, and new possibilities to journalism, it also raises ethical concerns.

Some key points from the article include:

1. Media outlets often rely on third-party solutions for AI-powered tools, which can lead to a lack of transparency and control over the models used.
2. Journalists' vision is crucial in preserving ethical values when integrating AI solutions into news production processes.
3. AI integration has led to changes in professional routines, journalist profiles, and final products, but it will not completely eliminate jobs; rather, certain roles may become redundant.
4. The use of generative AI poses questions about algorithmic bias, data privacy, human supervision, and the potential for misinformation.

The article also references various studies and reports on the 